# Car damage detection

CV model that distinguishes between whole and damaged cars.

## Setup

### Dependencies

Import the necessary modules.

* kagglehub: used to download the dataset.
* pathlib: provide path to the dataset.
* ImageDataLoaders: create a data block to be used in the CV model.
* Resize: used to transform all images in the dataset into one uniform size before passing to the model.
* aug_transforms: apply a set of default data augmentation transformations to the dataset. Helps with generalization of the model by introducing variations in training data.
* vision_learner: build a vision learner from the data object.
* resnet34: the model that will be used.
* error_rate: metric for the human to validate the model's performance.

In [2]:
from kagglehub import dataset_download

from pathlib import Path

from fastai.vision.all import (
    ImageDataLoaders,
    RandomResizedCrop,
    aug_transforms,
    vision_learner,
    resnet34,
    error_rate,
    ClassificationInterpretation
)

### Variables

1. Define the path to the dataset.

In [ ]:
path_string = dataset_download("anujms/car-damage-detection")
path = Path(path_string) / 'data1a'

2. Define the data block for training

In [4]:
data_block = ImageDataLoaders.from_folder(
    path,
    'training',
    'validation',
    seed=42,
    item_tfms=RandomResizedCrop(244, min_scale=0.5),
    batch_tfms=aug_transforms()
)

### Training

Pass the defined data block to the vision learner for training. Fine tune it with the argument 1 (meaning one epoch or one whole pass through the dataset).

In [ ]:
learner = vision_learner(data_block, resnet34, metrics=error_rate)
learner.fine_tune(6)

Display the batch of the data to confirm visually that the training was successful.

In [ ]:
data_block.show_batch()

### Interpretation

Interpreter can be used to visualize the results of training in a form of a confusion matrix.
The numbers outside of the diagonal show how many samples the model got wrong.

In [ ]:
interpreter = ClassificationInterpretation.from_learner(learner)
interpreter.plot_confusion_matrix()

This can be futher studied with the `plot_top_losses` method, which outputs the loss images with the caption: prediction/actual/loss/probability.

The model is wrong with its prediction when either it predicted wrong (prediction and actual don't match) and is confident (high probability) or it predicted right (prediction and actual match) but it's not confident (low probability).

In [ ]:
interpreter.plot_top_losses(17)

## Using

Pass the image to the model to categorize it being either damaged or whole.

In [ ]:
test = Path('.') / 'test' / '1.jpg'
learner.predict(test)